In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import yaml
import tqdm.notebook as tqdm
import soundfile
import numpy as np
import wandb
api = wandb.Api()

import gin
gin.enter_interactive_mode()

from thesis.notebook_util import audio_bytes_to_np, play_audio, specplot
from thesis.timbre_transfer_util import get_lufs, normalize_lufs

In [ ]:
models = [
    {"dataset": "Violin", "artifact_name": "0725-ddspae-2", "display_name": "DDSP-full"},
    {"dataset": "Violin", "artifact_name": "0725-ddspae-cnn-1-rt", "display_name": "DDSP-CNN"},
    {"dataset": "Violin", "artifact_name": "0809-ddspae-cnn-5-rt", "display_name": "DDSP-CNN-Tiny"},
    {"dataset": "Violin", "artifact_name": "0725-ddspae-cnn-1-rtq", "display_name": "DDSP-CNN"},
    {"dataset": "Violin", "artifact_name": "0809-ddspae-cnn-5-rtq", "display_name": "DDSP-CNN-Tiny"},
# ]

# models_trumpet = [
    {"dataset": "Trumpet", "artifact_name": "0805-ddspae", "display_name": "DDSP-full"},
    {"dataset": "Trumpet", "artifact_name": "0804-ddspae-cnn-3-rt", "display_name": "DDSP-CNN"},
    {"dataset": "Trumpet", "artifact_name": "0809-ddspae-cnn-4-rt", "display_name": "DDSP-CNN-Tiny"},
    {"dataset": "Trumpet", "artifact_name": "0804-ddspae-cnn-3-rtq", "display_name": "DDSP-CNN"},
    {"dataset": "Trumpet", "artifact_name": "0809-ddspae-cnn-4-rtq", "display_name": "DDSP-CNN-Tiny"},
]

In [ ]:
api = wandb.Api()

def load_models(models):
    for model in models:
        artifact = api.artifact(
            f"neural-audio-synthesis-thesis/nas-evaluation/eval-{model['artifact_name']}:latest"
        )
        model["run"] = artifact.logged_by()

        print(artifact.created_at, model["artifact_name"])

load_models(models)
# load_models(models_trumpet)

In [ ]:
SAMPLES_DIR = "/Users/vaclav/prog/thesis/data/wandb-samples"


def download_audio_samples(run, samples_dir=SAMPLES_DIR, name_must_contain="audio_both"):
    paths = []
    for file in tqdm.tqdm(run.files(), leave=False):
        if file.mimetype.startswith("audio"):
            if name_must_contain in file.name:
                path = os.path.join(SAMPLES_DIR, file.name)

                if os.path.isfile(path):
                    paths.append(path)
                else:
                    f = file.download(samples_dir, replace=False)
                    paths.append(f.name)
                    f.close()

    return paths

In [ ]:
for model in tqdm.tqdm(models):
    model["paths"] = download_audio_samples(model["run"])
    model["tt_paths"] = download_audio_samples(model["run"], name_must_contain="timbre_transfer")
    
# paths1 = download_audio_samples(run1)
# paths2 = download_audio_samples(run1)

In [ ]:
webmushra_dir = "/Users/vaclav/prog/webMUSHRA/"

max_clips = 15

def prepare_samples(models, subdir, paths_key="paths", included_indices=None):
    for model in models:
        assert len(model[paths_key]) == len(models[0][paths_key]), f"Different number of samples in {model['artifact_name']} {model['run'].id}"

#     out_dir = os.path.join(SAMPLES_DIR, "preprocessed", subdir)
    out_dir = os.path.join(webmushra_dir, "audio", subdir)
    
    if os.path.isdir(out_dir):
        for f in os.listdir(out_dir):
            if f.startswith("sample") and f.endswith(".wav"):
                os.remove(os.path.join(out_dir, f))
    
    os.makedirs(out_dir, exist_ok=True)

    processed_paths = []
    
    for j, cur_paths in tqdm.tqdm(list(enumerate(zip(*[p[paths_key] for p in models])))):
        if included_indices is not None and j not in included_indices:
            continue
        
        # audios = [[] for _ in range(len(paths_per_model) + 1)]
        audio_length = None
        audio_gt = None
        lufs = None
        skip = False
        cur_processed_paths = []
        
        for i, path in enumerate(cur_paths):
            with open(path, "rb") as f:
                audio_cur = audio_bytes_to_np(f.read(), normalize_db=None)

                if not audio_length:
                    audio_length = len(audio_cur)

                audio_pred_cur = audio_cur[:audio_length // 2]
                audio_gt_cur = audio_cur[audio_length // 2:]

                if audio_gt is None:
                    audio_gt = audio_gt_cur
                    # audios[-1].append(audio_gt)
                    lufs = get_lufs(audio_gt)
                    if lufs < -40:
                        print(f"Loudness is {lufs:.2f} LUFS for sample {j}, skipping")
                        skip = True
                    else:
                        audio_gt_cur = normalize_lufs(audio_gt_cur, target_lufs=-20, actual_lufs=lufs)
                        assert np.max(np.abs(audio_gt_cur)) <= 1.0
                        output_path = os.path.join(out_dir, f"sample_{j}_gt.wav")
                        soundfile.write(output_path, audio_gt_cur, samplerate=16000)
                        cur_processed_paths.append(output_path)
                else:
                    assert np.allclose(audio_gt_cur, audio_gt)

            if skip:
                break
            
            audio_pred_cur = normalize_lufs(audio_pred_cur, target_lufs=-20)
            output_path = os.path.join(out_dir, f"sample_{j}_{models[i]['artifact_name']}.wav")
            soundfile.write(output_path, audio_pred_cur, samplerate=16000)
            cur_processed_paths.append(output_path)
        
        if not skip:
            processed_paths.append(cur_processed_paths)
            if len(processed_paths) == max_clips:
                break
    
    return processed_paths

violin_processed_paths = prepare_samples(models[:5], subdir="violin")
trumpet_processed_paths = prepare_samples(models[5:], subdir="trumpet")

In [ ]:
tt_indices = [
    2,  # guitar (for training)
    8,  # clarinet
#     10,  # voice (m)
    11,  # flute
    13,  # voice (m)
    20,  # sax
    23,  # guitar
    29,  # voice (f)
]
violin_tt_processed_paths = prepare_samples(
    models[:5], subdir="violin_tt", paths_key="tt_paths", included_indices=tt_indices,
)
trumpet_tt_processed_paths = prepare_samples(
    models[5:], subdir="trumpet_tt", paths_key="tt_paths", included_indices=tt_indices,
)

- type: mushra
            id: trial_random_1
            name: MUSHRA - Random 2
            content: something something
            enableLooping: true 
            reference: configs/resources/audio/mono_ref.wav
            createAnchor35: false
            createAnchor70: false
            stimuli:
                C1: configs/resources/audio/mono_c1.wav
                C2: configs/resources/audio/mono_c2.wav
                C3: configs/resources/audio/mono_c3.wav          

In [ ]:
page_config_template = {
    "type": "mushra",
    "id": None,
    "content": "Please rate the naturalness of all conditions by comparing them to the reference.",
    "enableLooping": True,
    "reference": None,
    "createAnchor35": True,
    "createAnchor70": False,  # Limited by sample rate anyways
    "stimuli": [],
}

def create_page_config(processed_paths, models, overrides=None):
    if overrides is None:
        overrides = {}

    stimuli_config = ["random"]

    for sample_paths in processed_paths:
        cur_yaml = page_config_template.copy()
        cur_yaml["id"] = (
            os.path.basename(os.path.dirname(sample_paths[0]))
            + "_"
            + os.path.basename(sample_paths[0])
        )
#         cur_yaml["content"] = "Neural audio synthesis sample"
        cur_yaml["reference"] = os.path.relpath(sample_paths[0], webmushra_dir)
    
        for k, v in overrides.items():
            cur_yaml[k] = v

        cur_yaml["stimuli"] = {}
        for i in range(len(models)):
            cur_yaml["stimuli"][models[i]["artifact_name"]] = os.path.relpath(sample_paths[i + 1], webmushra_dir)

        stimuli_config.append(cur_yaml)
    
    return stimuli_config

In [ ]:
def update_generated_yaml():
    webmushra_config_dir = os.path.join(webmushra_dir, "configs")
    template_filename = "thesis_template.yaml"
    with open(os.path.join(webmushra_config_dir, template_filename), "r") as f:
        webmushra_config = yaml.safe_load(f)


    n_samples = 3
    webmushra_config["pages"][webmushra_config["pages"].index("DUMMY_VIOLIN")] = create_page_config(
        violin_processed_paths[:n_samples*2:2], models[:5]
    )
    webmushra_config["pages"][webmushra_config["pages"].index("DUMMY_TRUMPET")] = create_page_config(
        trumpet_processed_paths[:n_samples*2:2], models[5:]
    )

    tt_description_template = (
        "Please rate the naturalness of all conditions. "
        "<b>The conditions are supposed sound like {}.</b> "
        "<br>Due to technical limitations, the reference sound is also included among the conditions. You may leave it ungraded."
    )
    
    webmushra_config["pages"][webmushra_config["pages"].index("DUMMY_VIOLIN_TT")] = create_page_config(
        violin_tt_processed_paths, models[:5],
        overrides={
            "createAnchor35": False,
            "content": tt_description_template.format("a violin")
        },
    )
#     webmushra_config["pages"][webmushra_config["pages"].index("DUMMY_TRUMPET_TT")] = create_page_config(
#         trumpet_tt_processed_paths, models[5:],
#         overrides={
#             "createAnchor35": False,
#             "content": tt_description_template.format("a trumpet")
#         },
#     )

    data_str = yaml.safe_dump(webmushra_config)

    # To deal with parsing bug in webmushra
    data_str = data_str.replace("\n- - random\n", "\n-\n  - random\n")

    with open(os.path.join(webmushra_config_dir, "thesis_generated.yaml"), "w") as f:
        f.write(data_str)

update_generated_yaml()

In [ ]:
import time

while True:
    update_generated_yaml()
    time.sleep(2)